In [1]:
from dash import Dash, dash_table, dcc, html
from dash.dependencies import Input, Output
import pandas as pd
from flask import Flask
from Librerias_SAP import SAP_GUI, Funtions
import pandas as pd
import re
from collections import OrderedDict
from dash import dash_table as dt
from dash import dcc
from dash import html
from dash.dependencies import Input
from dash.dependencies import Output
import numpy as np
import dash_bootstrap_components as dbc
from datetime import datetime, timedelta

In [2]:
"""----------------------------Inciar session----------------------------------------------------"""
# Insert User name and password
Keys=pd.read_excel("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Claves\\Keys.xlsx")
user=Keys["User"][0]
password=Keys["Password"][0]
# Initialize session
session=SAP_GUI.SessionSAP(user,password)

In [3]:
# Disposiciones ZSD110
Name_ZSD110_FIRME="Pendiente_Firme"  # Revisar suele no guardar
Name_ZSD110_MERCADEO="Pendiente_Mercadeo"  # Revisar suele no guardar
Name_ZSD110_TOTAL="Pendiente_Total"  # Revisar suele no guardar


default_column_Canal01_Pendientes=['Can.distr.', 'Denomin.', 'Seccion', 'Estado', 'Solic.', 'Nombre',
       'Pedido', 'Clase Pedi', 'Creado el', 'Pedido.1', 'Posición', 'Material',
       'Vlr.Neto P', 'Moneda', 'Cantidad P', 'Cantidad C', 'UM venta',
       'Vlr.Neto C', 'Status Glo', 'Status Tot', 'Status Ent', 'Status Ent.1',
       'Status Ent.2']  


default_column_Canal01_Entregado=['Can.distr.', 'Denomin.', 'Seccion', 'Estado', 'Solic.', 'Nombre',
       'Entrega', 'Posicion E', 'F.Creacion', 'Pedido', 'Clase Pedi',
       'F.Creacion.1', 'Pedido.1', 'Posicion P', 'Material', 'Moneda',
       'Cantidad E', 'UM venta', 'Valor Neto', 'StTotPick.', 'StatGlWM',
       'StTotMovMe', 'Stat.fact.', 'Status PT']


default_column_Pedidos=['OrgVt', 'CDis', 'BqEn', 'Solic.', 'PrimFeEntr', 'ÚltEntrega',
       'FePrefEnt.', 'ClVt', 'Valor neto', 'Mon.', 'Doc.venta', 'ST',
       'Solicitante', 'Nº pedido cliente', 'Descrip.breve', 'Nº de cliente 1',
       'Contador de pedidos', 'Dirección', 'Barrio', 'Población', 'Teléfono']

"""----------------------------------------------LEER ARCHIVOS--------------------------------"""
#Data Pendientes
Canal01_Pendiente=pd.read_csv("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Documentos 1\\SAP\\SAP GUI\\Canal01_PEN.txt",skiprows=1,delimiter="\t")
Canal01_Pendiente=Funtions.Clean_Columns(Canal01_Pendiente)
Canal01_Pendiente=Funtions.default_column(default_column_Canal01_Pendientes,Canal01_Pendiente)

#Dta Entregados
Canal01_Entregado=pd.read_csv("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Documentos 1\\SAP\\SAP GUI\\Canal01_ENT.txt",skiprows=1,delimiter="\t")
Canal01_Entregado=Funtions.Clean_Columns(Canal01_Entregado)
Canal01_Entregado=Funtions.default_column(default_column_Canal01_Entregado,Canal01_Entregado)


"""""-------------------------------------------Limpio los datos-----------------------------------"""
#Canal01_Pendiente['Vlr.Neto C'], Canal01_Pendiente['Cantidad P']=Funtions.Clean_Num_List(Canal01_Entregado['Vlr.Neto C'],Canal01_Entregado['Cantidad P'])
Canal01_Entregado['Valor Neto'], Canal01_Entregado['Cantidad E']=Funtions.Clean_Num_List(Canal01_Entregado['Valor Neto'],Canal01_Entregado['Cantidad E'])


In [4]:
"""-----------------Datos descriptivos de interes-----------------------------------"""
#Lineas Totales
Lines_Canal01_Pendiente=len(Canal01_Pendiente)
Lines_Canal01_Entregado=len(Canal01_Entregado)

In [5]:
"""------------------------Optener datos de Agenda_Exito y Cencosub------------------"""
#Agenda Exito y Cencosub
Agenda_Exito=["0085","0020","0146","0149","0050","0138","0045"]
Agenda_Cencosub=["93","122","127","95","60"]


"""-----------------------------Aplico filtro en Agenda exito----------------------------------"""
Filtro_Agenda_Exito=Funtions.Datos_Agenda(Canal01_Entregado['Pedido.1'],Agenda_Exito)
Filtro_Agenda_Cencosub=Funtions.Search_agenda(Canal01_Entregado["Pedido.1"],Agenda_Cencosub)


"""------------------------------Canal 01 Entrega con y sin agenda Exito-----------------------"""
Filtro_Canal01_Entregado_Exito=(Canal01_Entregado['Pedido.1'].isin(Filtro_Agenda_Exito))   #Excluyo con ~
Canal01_Entregado_Agenda_Exito=Canal01_Entregado.loc[Filtro_Canal01_Entregado_Exito].reset_index(drop=True)
Canal01_Entregado_Sin_Agenda_Exito=Canal01_Entregado.loc[~Filtro_Canal01_Entregado_Exito].reset_index(drop=True)


"""------------------------------Canal 01 Entrega con y sin agenda Cencosub-----------------------"""
Filtro_Canal01_Entregado_Cencosub=(Canal01_Entregado['Pedido.1'].isin(Filtro_Agenda_Cencosub))   #Excluyo con ~
Canal01_Entregado_Agenda_Cencosub=Canal01_Entregado.loc[Filtro_Canal01_Entregado_Cencosub].reset_index(drop=True)
Canal01_Entregado_Sin_Agenda_Cencosub=Canal01_Entregado.loc[~Filtro_Canal01_Entregado_Cencosub].reset_index(drop=True)


In [6]:
#Creo las tablas dinamicas de cliente Exito y Cencosub

Table_dinamica_Exito_entrega=Canal01_Entregado_Agenda_Exito.pivot_table(index=['Pedido.1','Clase Pedi'],aggfunc={'Material':'count','Cantidad E':sum,'Valor Neto':sum}).reset_index()
Lineas_table_dinamica_Exito_entrega=sum(Table_dinamica_Exito_entrega["Material"])


Table_dinamica_Cencosub_entrega=Canal01_Entregado_Agenda_Cencosub.pivot_table(index=['Pedido.1','Clase Pedi'],aggfunc={'Material':'count','Cantidad E':sum,'Valor Neto':sum}).reset_index()
Lineas_table_dinamica_Cencosub_entrega=sum(Table_dinamica_Cencosub_entrega["Material"])

In [7]:
pedidos_Exito_Cencosub=pd.concat([Table_dinamica_Cencosub_entrega['Pedido.1'],Table_dinamica_Exito_entrega['Pedido.1']])

#Busco en la tansaccion ZSD037 los pedidos de las tablas dinamicas tanto de Exito y Cencosub

Transsaccion_ZSD037="zsd037"
SAP_GUI.Search_Pedidos_ZSD037(Transsaccion_ZSD037,pedidos_Exito_Cencosub,session)

In [8]:
Name_ZSD037="Pedidos pendientes Exito y Cencosub"
Ruta_ZSD037_Exito_Cencosub="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"
SAP_GUI.Export_TXT2(Name_ZSD037,session,Ruta_ZSD037_Exito_Cencosub)

In [9]:
# ZSD110 FIRME
url_ZSD037_Pedidos_Exito_Cencosub="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"+str(Name_ZSD037)+".txt"

Data_ZSD110_Pedidos_Exito_Cencosub=pd.read_csv(url_ZSD037_Pedidos_Exito_Cencosub,skiprows=5,delimiter="\t")
Data_ZSD110_Pedidos_Exito_Cencosub=Funtions.Clean_Columns(Data_ZSD110_Pedidos_Exito_Cencosub)
Data_ZSD110_Pedidos_Exito_Cencosub=Funtions.default_column(default_column_Pedidos,Data_ZSD110_Pedidos_Exito_Cencosub)
Data_ZSD110_Pedidos_Exito_Cencosub=Data_ZSD110_Pedidos_Exito_Cencosub[Data_ZSD110_Pedidos_Exito_Cencosub['Nº pedido cliente'].notnull()]



In [10]:
Data_ZSD110_Pedidos_Exito_Cencosub.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4940 entries, 0 to 4939
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   OrgVt                4940 non-null   float64
 1   CDis                 4940 non-null   float64
 2   BqEn                 0 non-null      float64
 3   Solic.               4940 non-null   float64
 4   PrimFeEntr           4940 non-null   object 
 5   ÚltEntrega           4940 non-null   object 
 6   FePrefEnt.           4940 non-null   object 
 7   ClVt                 4940 non-null   object 
 8   Valor neto           4940 non-null   object 
 9   Mon.                 4940 non-null   object 
 10  Doc.venta            4940 non-null   float64
 11  ST                   4940 non-null   object 
 12  Solicitante          4940 non-null   object 
 13  Nº pedido cliente    4940 non-null   object 
 14  Descrip.breve        4940 non-null   object 
 15  Nº de cliente 1      4417 non-null   o

In [11]:
try:
    Data_ZSD110_Pedidos_Exito_Cencosub['Nº pedido cliente']=SAP_GUI.Clean_int_to_str(Data_ZSD110_Pedidos_Exito_Cencosub['Nº pedido cliente'])   #OrgVt   #CDis  #4940
except:
    None  #No ese aplica metódo

In [12]:
#Dataframe de interes
Data_ZSD110_Pedidos_Pendientes_Fecha=Data_ZSD110_Pedidos_Exito_Cencosub[['Nº pedido cliente','PrimFeEntr','ÚltEntrega']].drop_duplicates()


In [13]:
#---------------------------------Tabla dinamica Exito---------------------------------------------
Tabla_Agenda_Exito=pd.merge(Data_ZSD110_Pedidos_Pendientes_Fecha,Table_dinamica_Exito_entrega,how="right",left_on="Nº pedido cliente",right_on="Pedido.1")[['Nº pedido cliente', 'PrimFeEntr', 'ÚltEntrega',
       'Clase Pedi', 'Cantidad E', 'Material', 'Valor Neto']]

Agenda_name_Exito={"0085":"FUNZA","0020":"VEGAS","0146":"BARRANQUILLA","0149":"BUCARAMANGA","0050":"CALI","0138":"PEREIRA","0045":"SURTIMAYORISTAS"}

Tabla_Agenda_Exito["Plataforma"]=Tabla_Agenda_Exito["Nº pedido cliente"].apply(lambda x: Agenda_name_Exito[re.findall("("+"|".join(list(Agenda_name_Exito.keys()))+")",x)[0]] if re.findall("("+"|".join(list(Agenda_name_Exito.keys()))+")",x)!=[] else x)


In [14]:
#Dataframe de interes

Tabla_Agenda_Cencusub=pd.merge(Data_ZSD110_Pedidos_Pendientes_Fecha,Table_dinamica_Cencosub_entrega,how="right",left_on="Nº pedido cliente",right_on="Pedido.1")[['Nº pedido cliente', 'PrimFeEntr', 'ÚltEntrega',
       'Clase Pedi', 'Cantidad E', 'Material', 'Valor Neto']]

Agenda_name_Cencosub={"93-":"MEDELLIN","122-":"BARRANQUILLA","127-":"BUCARAMANGA","95-":"CALI","60-":"BOGOTA"}


Tabla_Agenda_Cencusub["Plataforma"]=Tabla_Agenda_Cencusub["Nº pedido cliente"].apply(lambda x: Agenda_name_Cencosub[re.findall("\d*-",x)[0]] if re.findall("\d*-",x)[0] in list(Agenda_name_Cencosub.keys()) else None)


In [15]:
Tabla_Agenda_Cencusub

,Nº pedido cliente,PrimFeEntr,ÚltEntrega,Clase Pedi,Cantidad E,Material,Valor Neto,Plataforma
0,122-439976,08.06.2023,08.06.2023,ZVMI,723.0,334,11598587.0,BARRANQUILLA
1,127-443841,08.06.2023,08.06.2023,ZVMI,1820.0,544,28247168.0,BUCARAMANGA
2,60-1158908,02.06.2023,02.06.2023,ZEDI,118.0,25,5119346.0,BOGOTA
3,60-1158916,02.06.2023,04.06.2023,ZEDI,12.0,1,378348.0,BOGOTA
4,60-1159026,05.06.2023,05.06.2023,ZVMI,6098.0,2524,93697730.0,BOGOTA
5,93-485553,08.06.2023,08.06.2023,ZVMI,1008.0,476,15120226.0,MEDELLIN
6,95-493513,08.06.2023,08.06.2023,ZVMI,1123.0,389,14674390.0,CALI


# Segunda parte bot

In [16]:
# Pendiente Firme
Name_ZSD110_FIRME="Pendiente_Firme"  # Revisar suele no guardar

# Pendiente Mercadeo
Name_ZSD110_MERCADEO="Pendiente_Mercadeo"  # Revisar suele no guardar

# Pendiente Total 
Name_ZSD110_TOTAL="Pendiente_Total"  # Revisar suele no guardar


In [17]:
# ZSD110 FIRME
url_ZSD110_FIRME="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Documentos 1\\SAP\\SAP GUI\\"+str(Name_ZSD110_FIRME)+".txt"
Data_ZSD110_FIRME=pd.read_csv(url_ZSD110_FIRME,skiprows=1,delimiter="\t")
Data_ZSD110_FIRME=Funtions.Clean_Columns(Data_ZSD110_FIRME)
Data_ZSD110_FIRME["Lineas"]=Funtions.Clean_Num_List(Data_ZSD110_FIRME["Lineas"])


# ZSD110 MERCADE0
url_ZSD110_MERCADEO="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Documentos 1\\SAP\\SAP GUI\\"+str(Name_ZSD110_MERCADEO)+".txt"
Data_ZSD110_MERCADEO=pd.read_csv(url_ZSD110_MERCADEO,skiprows=1,delimiter="\t")
Data_ZSD110_MERCADEO=Funtions.Clean_Columns(Data_ZSD110_MERCADEO)
Data_ZSD110_MERCADEO["Lineas"]=Funtions.Clean_Num_List(Data_ZSD110_MERCADEO["Lineas"])


# ZSD110 Pendiente Total
url_ZSD110_PENDIENTES="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Documentos 1\\SAP\\SAP GUI\\"+str(Name_ZSD110_TOTAL)+".txt"
Data_ZSD110_PENDIENTES=pd.read_csv(url_ZSD110_PENDIENTES,skiprows=1,delimiter="\t")
Data_ZSD110_PENDIENTES=Funtions.Clean_Columns(Data_ZSD110_PENDIENTES)
Data_ZSD110_PENDIENTES["Lineas"]=Funtions.Clean_Num_List(Data_ZSD110_PENDIENTES["Lineas"])

"""---------------------CAMBIAR INDICES------------------------------------------"""
Channels_Name={1:"MODERNO",2:"TRADICIONAL",7:"TIENDAS DE BELLEZA",8:"PROFESIONAL",12:"INSTITUCIONAL",14:"HARD DISCOUNTERS",15:"ECOMMERCE",19:"ALTERNATIVO",50:"ALMACENES INTERNOS"}

#Aplicar transformacion
Data_ZSD110_MERCADEO["Can.distr."]=Data_ZSD110_MERCADEO["Can.distr."].replace(Channels_Name)
Data_ZSD110_FIRME["Can.distr."]=Data_ZSD110_FIRME["Can.distr."].replace(Channels_Name)
Data_ZSD110_PENDIENTES["Can.distr."]=Data_ZSD110_PENDIENTES["Can.distr."].replace(Channels_Name)

In [18]:
Data_ZSD110_PENDIENTES

,Can.distr.,Pedidos,Lineas,Unidades,Valor Neto
0,MODERNO,336,913.0,"24,731","388,448,000"
1,TRADICIONAL,54,608.0,"21,122","208,136,200"
2,TIENDAS DE BELLEZA,107,1428.0,"11,730","138,367,500"
3,PROFESIONAL,81,803.0,"2,681","97,702,600"
4,INSTITUCIONAL,2,2.0,370,"4,477,200"
5,HARD DISCOUNTERS,0,0.0,0,0
6,ECOMMERCE,17,77.0,82,"1,517,100"
7,ALTERNATIVO,4,125.0,"26,703","169,478,200"
8,ALMACENES INTERNOS,0,0.0,0,0
9,NaN,601,3956.0,"87,419","1,008,126,800"


In [19]:
#Aplicar consolidado de todas las tablas Mercadeo Firme Total
Consolidado_Mercadeo_Firme_Total=pd.merge(pd.merge(Data_ZSD110_MERCADEO,Data_ZSD110_FIRME,how="right",on="Can.distr."),Data_ZSD110_PENDIENTES,how="right",on="Can.distr.")
Consolidado_Mercadeo_Firme_Total=Consolidado_Mercadeo_Firme_Total.fillna(0)

In [20]:

Column_Consolidado_Mercadeo_Firme_Total=['Can.distr.', 'Pedidos', 'Lineas', 'Unidades', 'Valor Neto',
       'Pedidos', 'Lineas', 'Unidades', 'Valor Neto', 'Pedidos',
       'Lineas', 'Unidades', 'Valor Neto']

Consolidado_Clientes=Funtions.default_column(Column_Consolidado_Mercadeo_Firme_Total,Consolidado_Mercadeo_Firme_Total)

In [21]:
#Creo tabla total de la linea de clientes

Datos_lineas_clientes=pd.DataFrame([[1,"ECOMMERCE",0],[2,"Cencosud",0],[3,"Éxito Agenda",0],
              [4,"Canal 1",0],[5,"Canales 2,7,8",0],[6,"BAZAR",0],[7,"Mercadeo",0],
              [8,"Canales 14, 19 Citas",0],[9,"Extra agenda",0]],columns=["Prioridad","Clientes","Cantidad"])

In [22]:
#Calclulo las celdas calculadas

valor_ecommerce =Consolidado_Clientes.loc[6][6]   #6 Ecommers  y 6 Lineas
valor_canal_2_7_8=Consolidado_Clientes.loc[1][6]+Consolidado_Clientes.loc[2][6]+Consolidado_Clientes.loc[3][6]   #Tradiconal suma(1,2,3)
valor_Bazar=Consolidado_Clientes.loc[4][6]   
valor_Mercadeo=sum([Consolidado_Clientes.loc[i][2] for i in range(0,len(Consolidado_Clientes)-1)])
valor_canal_14_19=Consolidado_Clientes.loc[5][6]+Consolidado_Clientes.loc[7][6]

In [23]:
#Remplazo valores en las celdas calculadas
Datos_lineas_clientes["Cantidad"][0] = valor_ecommerce
Datos_lineas_clientes["Cantidad"][4] = valor_canal_2_7_8
Datos_lineas_clientes["Cantidad"][5] = valor_Bazar
Datos_lineas_clientes["Cantidad"][6] = valor_Mercadeo
Datos_lineas_clientes["Cantidad"][7] = valor_canal_14_19

C:\Users\prac.ingindustrial2\AppData\Local\Temp\ipykernel_10180\602017340.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Datos_lineas_clientes["Cantidad"][0] = valor_ecommerce
C:\Users\prac.ingindustrial2\AppData\Local\Temp\ipykernel_10180\602017340.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Datos_lineas_clientes["Cantidad"][4] = valor_canal_2_7_8
C:\Users\prac.ingindustrial2\AppData\Local\Temp\ipykernel_10180\602017340.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

In [24]:
Datos_lineas_clientes   #Total lineas del total - lineas cencosub - lineas exito - canal 1

,Prioridad,Clientes,Cantidad
0,1,ECOMMERCE,98
1,2,Cencosud,0
2,3,Éxito Agenda,0
3,4,Canal 1,0
4,5,"Canales 2,7,8",2659
5,6,BAZAR,2
6,7,Mercadeo,177
7,8,"Canales 14, 19 Citas",125
8,9,Extra agenda,0


# Parte 4

In [25]:
transsaccion="ZSD035D"
variant="JESPINOSAP"
provision="INFORME FACTU"

In [26]:

#SAP_GUI.Search_ZSD035D(transsaccion,session,provision,"15.06.2023",variant)

In [33]:
Un_dia_atras=(datetime.now()-timedelta(days=1)).date()
Un_dia_atras=Funtions.stand_day(Un_dia_atras)
SAP_GUI.Search_ZSD035D(transsaccion,session,provision,Un_dia_atras,variant)
# Pendiente Firme
Name_ZSD035D="Pendiente_Clientes"  # Revisar suele no guardar
SAP_GUI.Export_TXT2(Name_ZSD035D,session)

In [38]:
# Pendiente Firme
url_ZSD035D="C:\\Users\\\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Documentos 1\\SAP\\SAP GUI\\Archivos_CSV\\"+str(Name_ZSD035D)+".txt"
Data_ZSD035D=pd.read_csv(url_ZSD035D,skiprows=5,delimiter="\t")
Data_ZSD035D=Funtions.Clean_Columns(Data_ZSD035D)
Data_ZSD035D=Data_ZSD035D[Data_ZSD035D['Material'].notnull()]
#Data_ZSD035D["Lineas"]=Data_ZSD035D["Lineas"].apply(lambda x:Funtions.Clean_num(x))    
#Tener encuenta esta lineas puede este en dato numerico

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\\\Users\\\\prac.ingindustrial2\\\\OneDrive - Prebel S.A\\\\Documentos 1\\\\SAP\\\\SAP GUI\\\\Archivos_CSV\\\\Pendiente_Clientes.txt'

In [45]:
url_ZSD035D=r"C:\Users\prac.ingindustrial2\OneDrive - Prebel S.A\Escritorio\SAP\Archivos_CSV\Pendiente_Clientes.txt"
data = pd.read_csv(url_ZSD035D, delimiter="\t")

data

In [43]:
url_ZSD035D=r"C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Documentos 1\\SAP\\SAP GUI\\"+str(Name_ZSD035D)+".txt"
Data_ZSD035D=pd.read_csv(url_ZSD035D,skiprows=7,delimiter="\t")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\\\Users\\\\prac.ingindustrial2\\\\OneDrive - Prebel S.A\\\\Documentos 1\\\\SAP\\\\SAP GUI\\\\Pendiente_Clientes.txt'